# Анализ интернет-магазина по продаже продуктов питания

## Загрузим данные и подготовим их к анализу.

### Импортируем нужные модули.

In [1]:
import pandas as pd
import os
import numpy as np
from datetime import datetime, timedelta
import seaborn as sns
from matplotlib import pyplot as plt
import matplotlib.pyplot as plt
import scipy.stats as stats
import warnings
warnings.filterwarnings('ignore')

### Загрузим данные.

1. C помощью библиотеки os и проверкой на правильности пути файла, загрузим данные с помощью функции read_csv().

In [2]:
path = '/datasets/logs_exp.csv'

if os.path.exists(path):
    data = pd.read_csv(path, sep='\t')
else:
    print('Something is wrong')

2. Выведем для наглядности всю таблицу.

In [3]:
data.head(10)

,EventName,DeviceIDHash,EventTimestamp,ExpId
0,MainScreenAppear,4575588528974610257,1564029816,246
1,MainScreenAppear,7416695313311560658,1564053102,246
2,PaymentScreenSuccessful,3518123091307005509,1564054127,248
3,CartScreenAppear,3518123091307005509,1564054127,248
4,PaymentScreenSuccessful,6217807653094995999,1564055322,248
5,CartScreenAppear,6217807653094995999,1564055323,248
6,OffersScreenAppear,8351860793733343758,1564066242,246
7,MainScreenAppear,5682100281902512875,1564085677,246
8,MainScreenAppear,1850981295691852772,1564086702,247
9,MainScreenAppear,5407636962369102641,1564112112,246


3. Выведем основную информацию о столбцах таблицы.

In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 244126 entries, 0 to 244125
Data columns (total 4 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   EventName       244126 non-null  object
 1   DeviceIDHash    244126 non-null  int64 
 2   EventTimestamp  244126 non-null  int64 
 3   ExpId           244126 non-null  int64 
dtypes: int64(3), object(1)
memory usage: 7.5+ MB


**Вывод:** после выгрузки файла можем сделать вывод, что таблица состоит из четырех полей и 244126 записей. Уже можем заметить, что названия столбцов написаны неверно, столбец 'EventTimestamp' стоит привести в формат даты и времени, датафрейм стоит проверить на наличие пропусков и дубликатов.

### Предобработка данных.

1. Заменим название столбцов на правильный формат. Сделаем это с помощью метода rename().

In [5]:
data = data.rename(columns={'EventName':'event_name', 'DeviceIDHash':'device_id_hash','EventTimestamp':'event_timestamp','ExpId':'exp_id'})

In [6]:
data

,event_name,device_id_hash,event_timestamp,exp_id
0,MainScreenAppear,4575588528974610257,1564029816,246
1,MainScreenAppear,7416695313311560658,1564053102,246
2,PaymentScreenSuccessful,3518123091307005509,1564054127,248
3,CartScreenAppear,3518123091307005509,1564054127,248
4,PaymentScreenSuccessful,6217807653094995999,1564055322,248
...,...,...,...,...
244121,MainScreenAppear,4599628364049201812,1565212345,247
244122,MainScreenAppear,5849806612437486590,1565212439,246
244123,MainScreenAppear,5746969938801999050,1565212483,246
244124,MainScreenAppear,5746969938801999050,1565212498,246


2. Проверим датафрейм на наличие дубликатов.

In [7]:
data.duplicated().sum()

413

**Вывод:** в датафрейме 413 дубликатов, не такое большое количество, поэтому можем их удалить.

3. С помощью функции drop_duplicates() удаляем дубликаты из датафрейма.

In [8]:
data = data.drop_duplicates()

4. Проверим таблица на наличие пропусков.

In [9]:
data.isna().sum()

event_name         0
device_id_hash     0
event_timestamp    0
exp_id             0
dtype: int64

**Вывод:** в датафрейме пропуски отсутствуют.

### Добавление новых столбцов в таблицу.

1. Добавим новый столбец, в котором будет показана дата совершенной операции.

In [10]:
data['event_datetime'] = pd.to_datetime(data['event_timestamp'], unit='s').dt.strftime('%Y-%m-%d %H:%M:%S')

In [11]:
data

,event_name,device_id_hash,event_timestamp,exp_id,event_datetime
0,MainScreenAppear,4575588528974610257,1564029816,246,2019-07-25 04:43:36
1,MainScreenAppear,7416695313311560658,1564053102,246,2019-07-25 11:11:42
2,PaymentScreenSuccessful,3518123091307005509,1564054127,248,2019-07-25 11:28:47
3,CartScreenAppear,3518123091307005509,1564054127,248,2019-07-25 11:28:47
4,PaymentScreenSuccessful,6217807653094995999,1564055322,248,2019-07-25 11:48:42
...,...,...,...,...,...
244121,MainScreenAppear,4599628364049201812,1565212345,247,2019-08-07 21:12:25
244122,MainScreenAppear,5849806612437486590,1565212439,246,2019-08-07 21:13:59
244123,MainScreenAppear,5746969938801999050,1565212483,246,2019-08-07 21:14:43
244124,MainScreenAppear,5746969938801999050,1565212498,246,2019-08-07 21:14:58


2. Добавим новый столбец, в котором будет показана дата совершенной операции.

In [12]:
data['event_date'] = pd.to_datetime(data['event_timestamp'], unit='s').dt.strftime('%Y-%m-%d')

In [13]:
data

,event_name,device_id_hash,event_timestamp,exp_id,event_datetime,event_date
0,MainScreenAppear,4575588528974610257,1564029816,246,2019-07-25 04:43:36,2019-07-25
1,MainScreenAppear,7416695313311560658,1564053102,246,2019-07-25 11:11:42,2019-07-25
2,PaymentScreenSuccessful,3518123091307005509,1564054127,248,2019-07-25 11:28:47,2019-07-25
3,CartScreenAppear,3518123091307005509,1564054127,248,2019-07-25 11:28:47,2019-07-25
4,PaymentScreenSuccessful,6217807653094995999,1564055322,248,2019-07-25 11:48:42,2019-07-25
...,...,...,...,...,...,...
244121,MainScreenAppear,4599628364049201812,1565212345,247,2019-08-07 21:12:25,2019-08-07
244122,MainScreenAppear,5849806612437486590,1565212439,246,2019-08-07 21:13:59,2019-08-07
244123,MainScreenAppear,5746969938801999050,1565212483,246,2019-08-07 21:14:43,2019-08-07
244124,MainScreenAppear,5746969938801999050,1565212498,246,2019-08-07 21:14:58,2019-08-07


3. Выведем общую информацию о таблице.

In [15]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 243713 entries, 0 to 244125
Data columns (total 6 columns):
 #   Column           Non-Null Count   Dtype 
---  ------           --------------   ----- 
 0   event_name       243713 non-null  object
 1   device_id_hash   243713 non-null  int64 
 2   event_timestamp  243713 non-null  int64 
 3   exp_id           243713 non-null  int64 
 4   event_datetime   243713 non-null  object
 5   event_date       243713 non-null  object
dtypes: int64(3), object(3)
memory usage: 13.0+ MB


**Вывод:** тип данных новых столбцов с датами: 'object', необходимо их заменить в тип данных времени.

4. Воспользуемся методом astype() и изменим тип данных с 'object' на 'datetime64'.

In [20]:
data['event_datetime'] = data['event_datetime'].astype('datetime64[ns]')
data['event_date'] = data['event_date'].astype('datetime64[ns]')

In [21]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 243713 entries, 0 to 244125
Data columns (total 6 columns):
 #   Column           Non-Null Count   Dtype         
---  ------           --------------   -----         
 0   event_name       243713 non-null  object        
 1   device_id_hash   243713 non-null  int64         
 2   event_timestamp  243713 non-null  int64         
 3   exp_id           243713 non-null  int64         
 4   event_datetime   243713 non-null  datetime64[ns]
 5   event_date       243713 non-null  datetime64[ns]
dtypes: datetime64[ns](2), int64(3), object(1)
memory usage: 13.0+ MB


**Общий вывод по разделу:** в данном разделе мы загрузили основной датафрейм, предобработали его: изменили названия столбцов на правильный формат, удалили дубликаты, проверили на пропуски, которых не было. Добавили два новых столбца: с датой и временем и датой, изменили . Датасет готовы для дальнейшего анализа.